In [1]:
import os
import sys
import random
from mysqlconnect import connect_to_mysql

connection = connect_to_mysql()
cursor = connection.cursor()

USER: hitit-user, HOST: hitit-db-mydata.c9oy8g6q0v76.ap-northeast-2.rds.amazonaws.com, PORT: 3306, DATABASE: hitit
Connected to MySQL database


In [2]:
def get_user_risk_class(user_id):
    query = f"select investment_style_class from user_style where user_id = {user_id}"
    cursor.execute(query)
    result = cursor.fetchone()
    return result[0]

def get_recommend_fund(risk_class):
    query = f"select fund_name from fund_products where risk_grade = {risk_class+1}"
    cursor.execute(query)
    result = cursor.fetchall()
    result = [elem[0] for elem in result]
    return result


#4번 유저 위험 가져오기
user_risk = get_user_risk_class(4)

#4번유저의 위험도 바탕으로 펀드 추천하기
fund_lists = get_recommend_fund(user_risk)

#5개 선택
random.sample(fund_lists, 5)

['신한공모주&밴드트레이딩30증권자투자신탁[채권혼합](종류C1)',
 'DB퇴직연금40증권자투자신탁[채권혼합]운용',
 '미래에셋삼바브라질연금저축증권자투자신탁[채권]',
 '삼성누버거버먼글로벌인컴증권자투자신탁H[채권-재간접형]_A',
 'KB가치배당40증권자투자신탁(채권혼합) A-I 클래스']

In [3]:
column_names = [
    'fund_code', 'fund_name', 'hashtag', 'std_price', 'set_date', 'fund_type',
    'fund_type_detail', 'set_amount', 'company_name', 'risk_grade', 'risk_grade_txt',
    'drv_nav', 'bond', 'bond_foreign', 'stock', 'stock_foreign', 'investment',
    'etc', 'return_1m', 'return_3m', 'return_6m', 'return_1y', 'return_3y',
    'return_5y', 'return_idx', 'return_ytd', 'arima_price', 'arima_update',
    'arima_percent'
]
level = 5

query = \
f"""WITH RankedProducts AS (
    SELECT *,
    ROW_NUMBER() OVER (PARTITION BY company_name ORDER BY arima_percent DESC) AS rn
    FROM fund_products_4
    WHERE risk_grade >= {level} AND company_name <> '신한자산운용'
)
SELECT *
FROM (
    SELECT *
    FROM RankedProducts
    WHERE rn = 1

    UNION ALL

    SELECT * , 1 as rn
    FROM fund_products_4
    WHERE risk_grade >= {level} AND company_name = '신한자산운용'
) AS CombinedResults
ORDER BY arima_percent DESC
LIMIT 5;
"""
cursor.execute(query)
fetched_data = cursor.fetchall()

data_dict = [dict(zip(column_names, row)) for row in fetched_data]
for elem in data_dict:
    print(elem)

{'fund_code': 'K55301BW2052', 'fund_name': '미래에셋헤지펀드셀렉션혼합자산자투자신탁(사모투자재간접형)', 'hashtag': '#해외혼합형#글로벌', 'std_price': 1492.37, 'set_date': datetime.date(2017, 9, 22), 'fund_type': '해외자산배분', 'fund_type_detail': '해외혼합형', 'set_amount': 513, 'company_name': '미래에셋자산운용', 'risk_grade': 5, 'risk_grade_txt': '낮은 위험', 'drv_nav': 766.16, 'bond': 0.0, 'bond_foreign': 0.0, 'stock': 0.02, 'stock_foreign': '0.00', 'investment': 90.81, 'etc': 9.17, 'return_1m': 3.03, 'return_3m': 6.92, 'return_6m': 10.37, 'return_1y': 14.71, 'return_3y': 20.56, 'return_5y': 37.2, 'return_idx': 53.33, 'return_ytd': 8.57, 'arima_price': 1537.1904108474687, 'arima_update': datetime.date(2024, 6, 20), 'arima_percent': 2.69}
{'fund_code': 'K55105B66518', 'fund_name': '삼성달러표시단기채권증권모투자신탁(USD)[채권]', 'hashtag': '#해외채권형#외화기준가제공펀드#모펀드', 'std_price': 14490.5, 'set_date': datetime.date(2015, 9, 10), 'fund_type': '글로벌채권', 'fund_type_detail': '해외채권형', 'set_amount': 527, 'company_name': '삼성자산운용', 'risk_grade': 5, 'risk_grade_txt': '낮은 위